In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import warnings
warnings.filterwarnings("ignore")

from hlpr_func import lat_mean, sindeg, calc_trend_HAC, get_weights_values

In [ ]:
datapath = "/glade/work/csinger/albedo/CFMIP2024_analysis/"

aqua = xr.open_dataset(datapath+"MODIS_AOD/Aqua_aod_hist.nc").rename_vars({"aod_mean":"aqua_aod_mean","aod_hist":"aqua_aod_hist"})
era = xr.open_dataset(datapath+"ERA5-CAMS/era5-interpolated.nc")[["landmask"]]
modis = xr.merge([aqua,era])

# select CALIOP period
modis = modis.sel(time=slice("2006-06","2020-07"))

# select SO region
ds = modis.sel(lat=slice(-75,-45)).where(modis.landmask==0)
# make fake time coord
ds = ds.assign_coords({"month_since_start": ("time", np.arange(len(ds.time.values)))})
# select DJF
ds = ds.where(ds.time.dt.month.isin([12,1,2]))
# remove austrailian bush fires (Dec 2019 - Mar 2020)
ds = ds.where(~(ds.time.dt.year.isin(2019) & ds.time.dt.month.isin(12)))
ds = ds.where(~(ds.time.dt.year.isin(2020) & ds.time.dt.month.isin([1,2])))
# drop landmask
ds

In [ ]:
# add weights
weights = ds.time.dt.days_in_month
weights = weights.where(weights.time.dt.month!=2, 28.65)
ds["days_in_month"] = weights

climatology = ds.groupby("time.month").mean("time")
deseasonalized = ds.groupby("time.month") - climatology + climatology.weighted(climatology.days_in_month).mean("month")
deseasonalized = ds.assign_coords({"month_since_start": ("time", np.arange(len(deseasonalized.time.values)))})

so = lat_mean(deseasonalized.mean("lon"))
da = so.aqua_aod_mean.dropna(dim="time")
t = da.month_since_start.values
y = da.values
_, trend, trend_pm = calc_trend_HAC(t, y, hac_maxlags=12)
print("aqua aod trend")
print(trend*120, trend_pm*120)

In [ ]:
modis_so = ds[["aqua_aod_hist"]]
modis_so

In [ ]:
def sample_from_data(hist_values, bins, n):
    bin_logmidpoints = np.exp((np.log(bins[:-1]) + np.log(bins[1:]))/2)
    cdf = np.cumsum(hist_values, axis=0)
    cdf = cdf / cdf[-1] # normalize

    nmax = np.max(n)
    random_from_cdf = np.nan * np.zeros((nmax, *np.shape(cdf)[1:]))
    for lati in np.arange(np.shape(cdf)[1]):
        for loni in np.arange(np.shape(cdf)[2]):
            if not np.isnan(np.sum(cdf[:,lati,loni])):
                values = np.random.rand(n[lati,loni])
                nvalues = len(values)
                value_bins = np.searchsorted(cdf[:,lati,loni], values)
                random_from_cdf[:nvalues,lati,loni] = bin_logmidpoints[value_bins]

    return random_from_cdf

def sample_aod_ts(ds_modis, sampling_factor=1200):
    bins = np.append(ds_modis.aod_bins, 10)
    aod_mean_sampled = np.zeros((len(ds_modis.time), len(ds_modis.lat), len(ds_modis.lon)))
    
    for ti in np.arange(len(ds_modis.time)):
        hist = ds_modis.isel(time=ti).aqua_aod_hist
        n_per_grid_float = hist.sum("aod_bins").values / sampling_factor
        n_per_grid = np.where(n_per_grid_float < 1, np.random.rand(1) < n_per_grid_float, np.round(n_per_grid_float)).astype(int)
        samples = sample_from_data(hist.values, bins, n_per_grid)
        
        aod_mean_sampled[ti,:,:] = np.nanmean(samples, axis=0)
    
    ds_sampled = xr.Dataset(
        data_vars = {
            "aod_mean_sampled": (("time","lat","lon"), aod_mean_sampled)
        },
        coords = {
            "time":ds_modis.time,
            "lat":ds_modis.lat,
            "lon":ds_modis.lon
        }
    )
    return ds_sampled

def sampled_aod_so_trend(ds_sampled):
    climatology = ds_sampled.groupby("time.month").mean("time")
    deseasonalized = ds_sampled.groupby("time.month") - climatology + climatology.mean("month")
    deseasonalized = deseasonalized.assign_coords({"month_since_start": ("time", np.arange(len(deseasonalized.time.values)))})

    so = lat_mean(deseasonalized.mean("lon"))
    da = so.aod_mean_sampled.dropna(dim="time")
    t = da.month_since_start.values
    w = get_weights_values(da)
    _, trend, trend_pm = calc_trend_HAC(t, da.values, w, hac_maxlags=12)
    
    return trend*120, trend_pm*120 # units = 1/decade

In [ ]:
nbootstrap = 3
sampling_factor = 1200
trend_bootstrap, SE_bootstrap = np.zeros(nbootstrap), np.zeros(nbootstrap)
for i in np.arange(nbootstrap):
    ds_sampled = sample_aod_ts(modis_so, sampling_factor)
    trend_bootstrap[i], SE_bootstrap[i] = sampled_aod_so_trend(ds_sampled)
    print(i, trend_bootstrap[i])

np.savetxt(datapath+f"MODIS_AOD/trend_weighted_bootstrap_n{nbootstrap}_freq{sampling_factor}_2006-2020.csv", trend_bootstrap, delimiter=',') 
np.savetxt(datapath+f"MODIS_AOD/error_weighted_bootstrap_n{nbootstrap}_freq{sampling_factor}_2006-2020.csv", SE_bootstrap, delimiter=',') 

In [ ]:
trend_bootstrap = np.loadtxt(datapath+"MODIS_AOD/trend_weighted_bootstrap_n100_freq1200_2006-2020.csv")

fig,ax = plt.subplots(1,1,figsize=(5,4))
plt.hist(trend_bootstrap, bins=20, color="k", alpha=0.3, label="''CALIOP-like''")

ax.axvline(-0.0159, color="k", alpha=0.8, lw=2, label="true CALIOP")
ax.axvline(0.006452, color="brown", lw=2, label="MODIS Aqua")

ax.legend(frameon=False, loc="best", fontsize=9)
ax.set_xlabel("Southern Ocean AOD trend [decade$^{-1}$]")

ax.set_yticks([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

# plt.savefig("paper-figures/bootstrap100_freq1200_aod_trends_2006-2020.png", dpi=200, facecolor="w", bbox_inches="tight")
plt.show()